<a href="https://colab.research.google.com/github/kenwkliu/ideas/blob/master/colab/llamaIndexReplicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama_index
!pip install -q replicate
!pip install -q docx2txt

# Get the Replicate API key
import os
from google.colab import userdata
os.environ["REPLICATE_API_TOKEN"] = userdata.get('replicate')

# download an doc to the data folder
!wget https://github.com/kenwkliu/ideas/raw/master/colab/data/hkuspaceSeminars.docx
!mkdir data
!mv hkuspaceSeminars.docx data

In [ ]:
from llama_index.llms import Replicate

'''
value = [0, 1]
A high temperature produces more unpredictable and creative results,
while a low temperature produces more deterministic and conservative output
'''
temperature = 0.1

llama2_7b_chat = "meta/llama-2-7b-chat:8e6975e5ed6174911a6ff3d60540dfd4844201974602551e10e9e87ab143d81e"
llm = Replicate(
    model=llama2_7b_chat,
    temperature=temperature,
    additional_kwargs={"top_p": 1, "max_new_tokens": 300},
)

In [ ]:
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import ServiceContext

# https://huggingface.co/BAAI/bge-small-en-v1.5
# FlagEmbedding can map any text to a low-dimensional dense vector which can be used for tasks
# like retrieval, classification, clustering, or semantic search.
# And it also can be used in vector databases for LLMs.
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

# llama index read the data folder, embed the content and store in a vector database
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

query_engine = index.as_query_engine()

In [ ]:
prompt = "What is the seminar on 27 Feb 2024"

print(query_engine.query(prompt))